<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Simple Linear Regression with Sacramento Real Estate Data

_Authors: Matt Brems, Sam Stack_

---

In this lab you will hone your exploratory data analysis (EDA) skills and practice constructing linear regressions on a data set of Sacramento real estate sales.  The data set contains information on qualities of the property, location of the property, and time of sale.

### 1. Read in the Sacramento housing data set.

The file is stored in the `datasets` subfolder.

In [196]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('./datasets/sacramento_real_estate_transactions.csv') 

### 2. Conduct exploratory data analysis on this data set. 

If you discover any issues with the data, fix them to the best of your ability. Report any notable findings.

_**Note:** Zip codes often have leading zeros — e.g., 02215 = Boston, MA — which will often get knocked off automatically by many software programs like Python or Excel._

In [9]:
df.shape

(985, 12)

In [6]:
df.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768


In [8]:
df.columns

Index(['street', 'city', 'zip', 'state', 'beds', 'baths', 'sq__ft', 'type',
       'sale_date', 'price', 'latitude', 'longitude'],
      dtype='object')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985 entries, 0 to 984
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   street     985 non-null    object 
 1   city       985 non-null    object 
 2   zip        985 non-null    int64  
 3   state      985 non-null    object 
 4   beds       985 non-null    int64  
 5   baths      985 non-null    int64  
 6   sq__ft     985 non-null    int64  
 7   type       985 non-null    object 
 8   sale_date  985 non-null    object 
 9   price      985 non-null    int64  
 10  latitude   985 non-null    float64
 11  longitude  985 non-null    float64
dtypes: float64(2), int64(5), object(5)
memory usage: 92.5+ KB


In [14]:
df['state'].unique()

array(['CA', 'AC'], dtype=object)

In [17]:
len(df['city'].unique())

39

In [22]:
len(df['zip'].unique())

68

In [25]:
df['beds'].unique()

array([2, 3, 1, 4, 0, 5, 8, 6])

In [27]:
df['baths'].unique()

array([1, 2, 3, 4, 0, 5])

In [33]:
len(df[df['sq__ft'] == 0])

171

In [93]:
df['type'].unique()

array(['Residential', 'Condo', 'Multi-Family', 'Unkown'], dtype=object)

In [96]:
df['sale_date'].unique()

array(['Wed May 21 00:00:00 EDT 2008', 'Tue May 20 00:00:00 EDT 2008',
       'Mon May 19 00:00:00 EDT 2008', 'Fri May 16 00:00:00 EDT 2008',
       'Thu May 15 00:00:00 EDT 2008'], dtype=object)

In [41]:
df['price'].value_counts().sort_values()

533000     1
99000      1
157788     1
149000     1
148500     1
          ..
250000     9
150000     9
170000     9
220000    10
4897      49
Name: price, Length: 605, dtype: int64

In [44]:
df[['latitude','longitude']].head()

,latitude,longitude
0,38.631913,-121.434879
1,38.478902,-121.431028
2,38.618305,-121.443839
3,38.616835,-121.439146
4,38.519470,-121.435768


In [49]:
df['latitude'].sort_values()

703   -121.503471
174     38.241514
372     38.242270
820     38.247659
63      38.251808
          ...    
833     38.935579
484     38.939802
142     38.945357
242     39.008159
686     39.020808
Name: latitude, Length: 985, dtype: float64

In [50]:
df['longitude'].sort_values()

310   -121.551704
445   -121.550527
117   -121.549521
318   -121.549437
446   -121.549049
          ...    
844   -120.693641
106   -120.604760
102   -120.603872
663   -120.597599
703     38.668433
Name: longitude, Length: 985, dtype: float64

## Data cleaning to do: 
 - 1) some state lables are AC and should be CA... but likely we will drop theis column. 
 - 2) sale date is the same for every part of the time except day of the week and date.... change it to the day of the week as an integer... In general I's like to change to epoch time. seconds since jan1 1970
 - 3) there are 171 houses with 0 square footage 
 - 4) there is a negative number in prices (index 703)
 - 5) there is a negative number in square footage (INDEX 703)
 - 6) one latitude is -121.503471 (index 703)
 - 7) one longitude is 38 ish ( index 703) 
 - 8)  ****there are 49 houses listed with the same freakishly low price: 4897 


In [55]:
df[df['state'] == 'AC']['state'] 

703    AC
Name: state, dtype: object

In [60]:
df['state'][703] = 'CA'

/var/folders/kl/2yh74fss7jj3r6_9sk1_4fmm0000gn/T/ipykernel_65726/3629027158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['state'][703] = 'CA'


In [151]:
df['state'].value_counts() # 1 done

CA    985
Name: state, dtype: int64

In [141]:
from datetime import datetime

In [143]:
#eg_timestring = 'Wed May 21 00:00:00 EDT 2008'

def date_to_day_number(s):
    '''takes in strings in format from data, 
    gives out integer for day of the week'''
    s = s.replace('EDT','MDT') # jerry rig because datetime does not recognize EDT
    orig_format =   "%a %b %d %H:%M:%S %Z %Y"
    date = datetime.strptime(s, orig_format)
    day_number = int(date.strftime('%w')) # %w =day of the week as an int between 0 and 6
    return day_number

#date_to_day_number('Wed May 21 00:00:00 EDT 2008')# a test case

3

In [156]:
try: # in case I have already deleted
    df['sale_day'] = df['sale_date'].apply(date_to_day_number)
    del df['sale_date']
except: pass

In [158]:
df['sale_day'].value_counts() #2 done

1    268
5    264
2    177
3    158
4    118
Name: sale_day, dtype: int64

In [162]:
# now who are these 171 houses with no square footage?
df[df['sq__ft']==0]['beds'].value_counts() # 108 of them have no bedrooms either... 

0    108
4     23
3     21
5     10
2      8
6      1
Name: beds, dtype: int64

In [165]:
# and all of those have no baths
df[(df['sq__ft']==0) & (df['beds'] == 0)]['baths'].value_counts() 

0    108
Name: baths, dtype: int64

The 108 with no bed bath or sq__ft might be empty lots, and are thus their data is correct. 

Those with bedrooms or bathrooms but no sq__ft are mistakes 

I should replace their sq__ft with nan to reduce their pull oin the linear regression 

In [189]:
no_b_filter = ((df['beds']==0) | (df['baths']==0))
no_sq_filter = (df['sq__ft']== 0)

In [191]:
(no_b_filter & no_sq_filter).sum()

108

In [201]:
# this compiles... but does not change
df.loc[no_b_filter & no_sq_filter, ['sq__ft','beds','baths'] ]  = np.nan

In [211]:
df[no_b_filter & no_sq_filter] # 3 done

,street,city,zip,state,beds,baths,sq__ft,type,price,latitude,longitude,sale_day
73,17 SERASPI CT,SACRAMENTO,95834,CA,NaN,NaN,NaN,Residential,206000,38.631481,-121.501880,3
89,2866 KARITSA AVE,SACRAMENTO,95833,CA,NaN,NaN,NaN,Residential,244500,38.626671,-121.525970,3
100,12209 CONSERVANCY WAY,RANCHO CORDOVA,95742,CA,NaN,NaN,NaN,Residential,263500,38.553867,-121.219141,3
121,5337 DUSTY ROSE WAY,RANCHO CORDOVA,95742,CA,NaN,NaN,NaN,Residential,320000,38.528575,-121.228600,3
126,2115 SMOKESTACK WAY,SACRAMENTO,95833,CA,NaN,NaN,NaN,Residential,339500,38.602416,-121.542965,3
...,...,...,...,...,...,...,...,...,...,...,...,...
901,1530 TOPANGA LN Unit 204,LINCOLN,95648,CA,NaN,NaN,NaN,Condo,138000,38.884150,-121.270277,4
917,501 POPLAR AVE,WEST SACRAMENTO,95691,CA,NaN,NaN,NaN,Residential,165000,38.584526,-121.534609,4
934,1550 TOPANGA LN Unit 207,LINCOLN,95648,CA,NaN,NaN,NaN,Condo,188000,38.884170,-121.270222,4
947,1525 PENNSYLVANIA AVE,WEST SACRAMENTO,95691,CA,NaN,NaN,NaN,Residential,200100,38.569943,-121.527539,4


In [214]:
df[df['price']<0]

,street,city,zip,state,beds,baths,sq__ft,type,price,latitude,longitude,sale_day
703,1900 DANBROOK DR,SACRAMENTO,95835,CA,1.0,1.0,-984.0,Condo,-210944,-121.503471,38.668433,5


In [224]:
ol_lat = df.loc[703,'latitude']
ol_lon = df.loc[703,'longitude'] 
ol_price = df.loc[703,'price'] 
ol_sf = df.loc[703, 'sq__ft']

In [229]:
if df.loc[703,'price'] <0: #just do this once
    df.loc[703,'latitude'] = ol_lon 
    df.loc[703,'longitude'] = ol_lat
    df.loc[703,'price'] = - ol_price
    df.loc[703, 'sq__ft'] = -ol_sf 

df.loc[703,:] # 4,5,6,7 done

street       1900 DANBROOK DR
city               SACRAMENTO
zip                     95835
state                      CA
beds                      1.0
baths                     1.0
sq__ft                  984.0
type                    Condo
price                  210944
latitude            38.668433
longitude         -121.503471
sale_day                    5
Name: 703, dtype: object

 - 8)  ****there are 49 houses listed with the same freakishly low price: 4897 

In [237]:
(df[df['price'] == 4897]) # 49, most of them with no square footage, 


,street,city,zip,state,beds,baths,sq__ft,type,price,latitude,longitude,sale_day
554,20 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.885327,-121.289412,1
555,24 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.885132,-121.289405,1
556,28 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.884936,-121.289397,1
557,32 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.884741,-121.289390,1
558,36 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.884599,-121.289406,1
559,40 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.884535,-121.289619,1
560,44 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.884590,-121.289835,1
561,48 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.884667,-121.289896,1
562,52 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.884780,-121.289911,1
563,68 CRYSTALWOOD CIR,LINCOLN,95648,CA,NaN,NaN,NaN,Residential,4897,38.885236,-121.289928,1


Looking at these houses on a map, and being familiar with the sacramento area, I am certain that this is junk data

In [243]:
df = df[df['price'] != 4897] # workaround because I'm failing to drop rows as expected

In [244]:
df.head()

,street,city,zip,state,beds,baths,sq__ft,type,price,latitude,longitude,sale_day
0,3526 HIGH ST,SACRAMENTO,95838,CA,2.0,1.0,836.0,Residential,59222,38.631913,-121.434879,3
1,51 OMAHA CT,SACRAMENTO,95823,CA,3.0,1.0,1167.0,Residential,68212,38.478902,-121.431028,3
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2.0,1.0,796.0,Residential,68880,38.618305,-121.443839,3
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2.0,1.0,852.0,Residential,69307,38.616835,-121.439146,3
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2.0,1.0,797.0,Residential,81900,38.519470,-121.435768,3


When you've finished cleaning or have made a good deal of progress cleaning, it's always a good idea to save your work. Use the pandas `to_csv` method to export your cleaned dataset.

```python
# example -- your dataframe name may be different
dataframe.to_csv('./datasets/sacramento_cleaned_dataframe.csv', index=False)

```

In [247]:
df.to_csv('./datasets/sacramento_cleaned_dataframe.csv', index=False)

### 3. Our goal will be to predict price. List variables that you think qualify as predictors of price in an SLR model.

For each of the variables you believe should be considered, generate a plot showing the relationship between the independent and dependent variables.

In [251]:
df.columns

Index(['street', 'city', 'zip', 'state', 'beds', 'baths', 'sq__ft', 'type',
       'price', 'latitude', 'longitude', 'sale_day'],
      dtype='object')

beds, baths, and sq__ft will be good predictors of price. 

In [ ]:
from sklearn.linear_model import LinearRegression  
y = df['price']
X = df[['sq__ft', 'beds','baths']]
lr = LinearRegression()
lr.fit(X,y) #ok.. now you tell me that nan can't go into lr... grr.... 

In [256]:
df

AttributeError: 'DataFrame' object has no attribute 'isnan'

In [255]:
lr.coef_

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### 4. Which variable would be the best predictor of Y in an SLR model? Why?

### 5. Write a function that will take in two lists, `x` and `y`, and return the intercept and slope coefficients that minimize SSE. 

`y` is the target variable, and `x` is the predictor variable.

**Don't use scikit-learn for this**. Instead, use numpy and scipy.

The slope and intercept of simple linear regression can be calculated by using certain summary statistics. In particular, we'll need to know:

- $\bar{x}$, the mean of `x`
- $\bar{y}$, the mean of `y`
- $s_{x}$, the standard deviation of `x`
- $s_{y}$, the standard deviation of `y`
- $r_{xy}$, the correlation of `x` and `y`

Given these summary statistics, the **coefficient on `X`** (or the slope) can be given by $\beta_{1} = r_{xy} * \frac{s_{y}}{s_{x}}$. The **intercept of the model** can be given by $\beta_{0} = \bar{y} - \beta_{1}\bar{x}$.

Your function should take in `x` and `y`, then use the above summary statistics to calculate the slope and intercept.

- **Test your function on price and the variable you determined was the best predictor in Problem 4.**
- **Report the slope and intercept.**

### 6. Interpret the intercept and the slope.

### 7. Give an example of how this model could be used for prediction and how it could be used for inference. 

**Be sure to make it clear which example is associated with prediction and which is associated with inference.**

### 8: [Bonus] Using the model you came up with in Problem 5, calculate and plot the residuals.

---

## Dummy Variables

It is important to be cautious with categorical variables, which represent distict groups or categories, when building a regression. If put in a regression "as-is," categorical variables represented as integers will be treated like *continuous* variables.

That is to say, instead of group "3" having a different effect on the estimation than group "1" it will estimate literally 3 times more than group 1. 

For example, if occupation category "1" represents "analyst" and occupation category "3" represents "barista", and our target variable is salary, if we leave this as a column of integers then barista will always have `beta*3` the effect of analyst.

This will almost certainly force the beta coefficient to be something strange and incorrect. Instead, we can re-represent the categories as multiple "dummy coded" columns.



#### A Word of Caution When Creating Dummies

Let's touch on precautions we should take when dummy coding.

**If you convert a qualitative variable to dummy variables, you want to turn a variable with N categories into N-1 variables.**

> **Scenario 1:** Suppose we're working with the variable "sex" or "gender" with values "M" and "F". 

You should include in your model only one variable for "sex = F" which takes on 1 if sex is female and 0 if sex is not female! Rather than saying "a one unit change in X," the coefficient associated with "sex = F" is interpreted as the average change in Y when sex = F relative to when sex = M.

| Female | Male | 
|-------|------|
| 0 | 1 | 
| 1 | 0 |
| 0 | 1 |
| 1 | 0 |
| 1 | 0 |
_As we can see a 1 in the female column indicates a 0 in the male column. And so, we have two columns stating the same information in different ways._

> Scenario 2: Suppose we're modeling revenue at a bar for each of the days of the week. We have a column with strings identifying which day of the week this observation occured in.

We might include six of the days as their own variables: "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday". **But not all 7 days.**  

|Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | 
|-------|---------|-----------|----------|--------|----------|
| 1     | 0       |0          |      0   |0       | 0        | 
| 0     | 1       |0          |      0   |0       | 0        | 
| 0     | 0       |1          |      0   |0       | 0        | 
| 0     | 0       |0          |      1   |0       | 0        | 
| 0     | 0       |0          |      0   |1       | 0        | 
| 0     | 0       |0          |      0   |0       | 1        | 
| 0     | 0       |0          |      0   |0       | 0        | 

_As humans we can infer from the last row that if its is not Monday, Tuesday, Wednesday, Thursday, Friday or Saturday, it must be Sunday. Models work the same way._

The coefficient for Monday is then interpreted as the average change in revenue when "day = Monday" relative to "day = Sunday." The coefficient for Tuesday is interpreted as the average change in revenue when "day = Tuesday" relative to "day = Sunday" and so on.

The category you leave out, which the other columns are *relative to*, is often referred to as the **reference category**.

### 9. Use the `pd.get_dummies` function to convert the `type` column into dummy-coded variables.

### 10. Check the dummy columns you've created. Do they all make sense?

Use `.head()` on the new dataframe you've created. Depending on how you cleaned the data earlier in this notebook, there may be a dummy column that needs to be removed. Check that all the columns you've created make sense, and if any of the columns seem to be errors, drop them. If there don't appear to be any errors, then move on to the next step!

### 11. Build what you think may be the best MLR model predicting `price`. 

Choose at least three variables, and build a model that uses them to predict price. At least one of the variables you choose should be a dummy-coded variable. (This can be one we created before or a new one.) 

You may need to load in the scikit-learn API:

```python
from sklearn.linear_model import LinearRegression

model = LinearRegression()
```

### 12. Plot the true price vs the predicted price to evaluate your MLR visually.

You can use matplotlib or seaborn.

### 13. List the five assumptions for an MLR model. 

Indicate which ones are the same as the assumptions for an SLR model. 

In [ ]:
# convert this cell to markdown to provide your answer

### 14. Pick at least two assumptions and explain whether or not you believe them to be met for your model, and explain why.

### 15. [Bonus] Generate a table showing the point estimates, standard errors, t-scores, p-values, and 95% confidence intervals for the model you built. 

**Write a few sentences interpreting some of the output.**

> **Hint:** scikit-learn does not have this functionality built in, but statsmodels does in the `summary` method.  To fit the statsmodels model use something like the following.  There is one big caveat here, however!  `statsmodels.OLS` does _not_ add an intercept to your model, so you will need to do this explicitly by adding a column filled with the number 1 to your X matrix

```python
import statsmodels.api as sm

# The Default here is Linear Regression (ordinary least squares regression OLS)
model = sm.OLS(y,X).fit()
```

### 16. Regression Metrics

Implement a function called `r2_adj()` that will calculate $R^2_{adj}$ for a model. 

### 17. Metrics, metrics, everywhere...

Calculate and interpret at least three regression metrics. How well does your model perform?

### 18. Model Iteration

Adjust your model by adding or removing a variable. Then calculate the same regression metrics as you used in question 17. Has your model performance increased or decreased?

### 19. Bias vs. Variance

_(To be completed after the bias-variance tradeoff lecture.)_ At this point, do you think your model is high bias, high variance or in the sweet spot?